# Customer Churn Prediction — Telecom Company
**Author:** : Ali Asadullah Shehbaz

**Project Type:** Machine Learning Classification 
 
**Objective:** Build a machine learning model to predict customer churn using historical data and provide actionable insights.


## 1. Introduction
Customer churn is when a customer stops using a company's service. In telecom, high churn directly impacts revenue and growth. This project builds a predictive model using historical customer data to identify churn risk.



## 2. Objective
- Perform Exploratory Data Analysis (EDA)
- Engineer features
- Train models (Logistic Regression, XGBoost etc)
- Evaluate performance using key metrics (confusion matrix, AUC-ROC)
- Present results with visualizations


## 3.Import Libraries

In [ ]:
# Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# Visualization settings
sns.set(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (8, 5)

# remove warnings
import warnings
warnings.filterwarnings('ignore')

## 4. Load Data
The dataset provided is a mock dataset simulating telecom customer records.  
It includes demographic, contract, and payment information along with a churn label.


In [ ]:
# Generate mock data
np.random.seed(42)
n = 10000
data = pd.DataFrame({
    'CustomerID': np.arange(n),
    'Gender': np.random.choice(['Male', 'Female'], size=n),
    'SeniorCitizen': np.random.choice([0, 1], size=n),
    'Tenure': np.random.randint(1, 72, size=n),
    'MonthlyCharges': np.round(np.random.uniform(20, 120, size=n), 2),
    'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], size=n),
    'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card'], size=n),
    'Churn': np.random.choice([0, 1], size=n, p=[0.73, 0.27])
})
data['TotalCharges'] = (data['Tenure'] * data['MonthlyCharges']).round(2)

# Quick check
data.head()
    

## 5. Exploratory Data Analysis (EDA)
The goal of EDA is to understand:
- Data shape, missing values, and types
- Distribution of target variable (Churn)
- Relationship between features and churn


In [ ]:
print(f"There are {data.shape[0]} rows and {data.shape[1]} columns in the data")

In [ ]:
print(f"The columns of the data are : {data.columns.tolist()}")

In [ ]:
# Dataset info
print(data.info())

**Observations**

1. **Shape & Size**
   - **10,000 rows** × **9 columns**.
   - Each row represents a unique customer (`CustomerID` from 0 to 9999).

2. **Missing Values**
   - **No missing values** detected across all columns.

3. **Data Types**
   - **Integer (`int64`, `int32`)**: `CustomerID`, `SeniorCitizen`, `Tenure`, `Churn`
   - **Float (`float64`)**: `MonthlyCharges`, `TotalCharges`
   - **Object (categorical)**: `Gender`, `Contract`, `PaymentMethod`

4. **Memory Usage**
   - Dataset occupies approximately **664 KB** — small enough for in-memory processing.

5. **Special Notes**
   - `TotalCharges` is derived from `Tenure × MonthlyCharges` and may be **highly correlated** with them.
   - All features are fully populated, so no imputation is required at this stage.


In [ ]:
print(data.isnull().sum()[data.isnull().sum() > 0])

In [ ]:
sns.heatmap(data.isnull(),cmap = 'magma',cbar = False)

In [ ]:
# Checking duplicates in the data 
data.duplicated('CustomerID').sum()

**Observations**
- No missing values detected.
- No duplicate `CustomerID` values.

In [ ]:
# Basic stats
data.describe().T

### 1. Univariate Analysis
We explore distributions of numerical and categorical features.


In [ ]:
data['Gender'].value_counts()

In [ ]:
sns.countplot(data=data, x='Gender')
plt.title('Gender Distribution')
plt.show()

**Observations**
- There are uniform distribution for gender column in the data with 5000 males and females

In [ ]:
data['SeniorCitizen'].value_counts()

In [ ]:
# SeniorCitizen distribution
sns.countplot(x='SeniorCitizen', data=data)
plt.title("SeniorCitizen Distribution")
plt.show()

**Observations**
- There are 50% citizens senior and 50% non-senior in the data indicating uniform distribution

In [ ]:
data['Tenure'].describe().T

In [ ]:
# Tenure Distribution
sns.histplot(x='Tenure', data=data)
plt.title('Tenure Distribution')
plt.show()

**Observation**
- The tenure distribution is fairly uniform across the range, with slight peaks around 5, 20, and 55 months, indicating customers join and leave consistently over time rather than clustering at specific contract lengths.

In [ ]:
data['MonthlyCharges'].describe().T

In [ ]:
sns.histplot(x='MonthlyCharges',data=data, kde=True)
plt.title('Monthly Charges Distribution')
plt.show()

**Observation**
- The MonthlyCharges variable appears to be uniformly distributed across its range, with relatively consistent counts in each interval.

In [ ]:
data['Contract'].value_counts()

In [ ]:
sns.countplot(x='Contract',data=data)
plt.title('Contract Distribution')
plt.show()

**Observation**
- The counts of customers are similar across all contract types, with no single type showing a significantly higher or lower number.

In [ ]:
data['PaymentMethod'].value_counts()

In [ ]:
sns.countplot(x='PaymentMethod',data=data)
plt.title('Payment Method Distribution')
plt.show()

**Observation**
- The number of customers using each payment method is nearly equal, indicating no strong preference for any specific payment option.

In [ ]:
data['Churn'].value_counts()

In [ ]:
# Churn distribution
sns.countplot(x='Churn', data=data)
plt.title("Churn Distribution")
plt.show()

In [ ]:
# Count churn and non-churn
churn_counts = data['Churn'].value_counts()

# Labels and colors
labels = ['No Churn', 'Churn']
colors = ['#66b3ff', '#ff9999']

# Plot
plt.figure(figsize=(6,6))
plt.pie(
    churn_counts,
    labels=labels,
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    explode=(0, 0.05),  # Slightly explode churn for emphasis
    shadow=True
)
plt.title('Customer Churn Distribution', fontsize=14)
plt.show()


**Observations**  
- The above plot shows that 73.3% People don't churn the bank and Only 26.7% People churn the bank 

In [ ]:
data['TotalCharges'].describe()

In [ ]:
# TotalCharges Distribution
sns.histplot(x='TotalCharges',data=data)
plt.title('Total Charges Distribution')
plt.show()

**Observations**

The TotalCharges distribution is right-skewed, with most customers clustered below $2,000, indicating:
- Many are low-spending or short-tenure customers.
- A long tail beyond $4,000 suggests a smaller segment of high-value, long-term customers.

### 2. Bivariate Analysis
We analyze how features differ between churned and non-churned customers.


In [ ]:
data.groupby('Gender')['Churn'].value_counts()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=data, x='Gender', hue='Churn')
plt.title('Churn Rate by Gender')
plt.show()

**Observations**
- The churn rates for Male and Female customers appear nearly identical, suggesting that gender has minimal influence on churn in this dataset.


In [ ]:
data.groupby('SeniorCitizen')['Churn'].value_counts()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=data, x='SeniorCitizen', hue='Churn')
plt.title('Churn Rate by Senior Citizen Status')
plt.show()

In [ ]:
data.groupby('Contract')['Churn'].value_counts()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=data, x='Contract', hue='Churn')
plt.title('Churn Rate by Contract Type')
plt.show()


**Observations**

- Customers on month-to-month contracts appear to have a higher proportion of churn compared to one-year and two-year contracts.

In [ ]:
data.groupby('PaymentMethod')['Churn'].value_counts()

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(data=data, x='PaymentMethod', hue='Churn')
plt.title('Churn Rate by Payment Method')
plt.xticks(rotation=45)
plt.show()

In [ ]:
data.groupby('MonthlyCharges')['Churn'].value_counts()

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(data=data, x='MonthlyCharges', hue='Churn')
plt.title('Churn Rate by Monthly Charges')
plt.show()

In [ ]:
# Monthly charges vs Churn
plt.figure(figsize=(7,5))
sns.boxplot(data=data, x='Churn', y='MonthlyCharges')
plt.title('Monthly Charges by Churn')
plt.show()

**Observation**
- Monthly Charges column is consistent for both conditions and identical 

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=data, x='Churn', y='Tenure')
plt.title('Tenure vs Churn')
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=data, x='Churn', y='TotalCharges')
plt.title('Total Charges vs Churn')
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(data=data, x='Tenure', y='MonthlyCharges', hue='Churn', alpha=0.5)
plt.title('Tenure vs Monthly Charges (Colored by Churn)')
plt.show()

In [ ]:
# Contract vs Payment Method vs Churn
pd.crosstab([data['Contract'], data['PaymentMethod']], data['Churn'], normalize='index') * 100

**Observations**
-  All the contract categories have the similar churn rate and all the payment methods have similar churn rate with minimal difference 

### 3. Correlation Analysis
We check correlation among numerical features and Churn.


In [ ]:
plt.figure(figsize=(8,6))
corr = data.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='Blues', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()


**Observations**
- Total Charges is 77% correlated with Tenure and 56% correlated with Monthly Charges 

### 4. Outlier Detection

In [ ]:
# Create box plots for each feature to visually inspect outliers
num_cols = ['Tenure', 'MonthlyCharges',
       'TotalCharges']

# Create box plots for each feature to visually inspect outliers
plt.figure(figsize=(15, 10))
for i, feature in enumerate(num_cols, 1):
    plt.subplot(2, 2, i)
    plt.boxplot(data[feature], vert=False)
    plt.title(f'Boxplot of {feature}')
    
plt.tight_layout()
plt.show()

## 6.Feature Engineering

Goal: create informative, non-leaking features and prepare a reproducible preprocessing pipeline for modeling.  
Plan:
1. Train/test split (stratified on target)
2. Basic cleaning & duplicate checks
3. Create derived features (tenure buckets, flags, interactions)
4. Encode categorical variables (One-Hot / Ordinal / Frequency)
5. Scale numeric features
6. Check multicollinearity (VIF / correlations) and drop/reduce redundancy
7. Save preprocessing pipeline (ColumnTransformer + pipeline)


### 1. Train/Test Split
Split data into train and test first to avoid leakage during feature engineering (especially for target/statistical encodings).
We use stratified splitting to preserve churn ratio in both sets.


In [ ]:
from sklearn.model_selection import train_test_split

# `data` is our DataFrame and 'Churn' is the target
X = data.drop(columns=['Churn', 'CustomerID'])   # drop ID from features
y = data['Churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


In [ ]:
X_train.head()

### 2. Basic Data Checks (train set)
Check for duplicates, invalid types, and missing values in the training set.


In [ ]:
# Check missing values and duplicates on train
print("Missing values per column (train):\n", X_train.isnull().sum())
print("Duplicate Customer rows (train):", X_train.duplicated().sum())
# dtype overview
print(X_train.dtypes)


### 3. Derived Features
Create features that capture business intuition:
- `TenureGroup` (binned tenure)
- `HighMonthlyFlag` (customers paying above median)
- `Contract_Ord` (ordinal mapping for contract length)
- `AutoPay` flag (bank transfer / credit card → likely auto-pay)
- `AvgChargesPerMonth` (TotalCharges / Tenure) — note: in this synthetic data it's identical to MonthlyCharges; we use it for demonstration and will handle redundancy later.
- `Contract_Payment` (combined category) to capture interaction


In [ ]:
# Work on training copy
train = X_train.copy()

# Tenure groups (example bins)
bins = [0, 3, 6, 12, 24, 48, 72]
labels = [
    'New (0-3 months)',
    'Early (4-6 months)',
    'Settling (7-12 months)',
    'Engaged (13-24 months)',
    'Loyal (25-48 months)',
    'Very Loyal (49-72 months)'
]
train['TenureGroup'] = pd.cut(train['Tenure'], bins=bins, labels=labels, include_lowest=True)
# Average charges per month
train['AvgChargesPerMonth'] = (train['TotalCharges'] / train['Tenure'])
# High monthly flag
median_charge = train['MonthlyCharges'].median()
train['HighMonthlyFlag'] = (train['MonthlyCharges'] > median_charge).astype(int)
# Contract ordinal
contract_map = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
train['Contract_Ord'] = train['Contract'].map(contract_map)

# Autopay flag (assumption: bank transfer & credit card => auto)
autopay_methods = ['Bank transfer', 'Credit card']
train['AutoPay'] = train['PaymentMethod'].isin(autopay_methods).astype(int)

# Interaction feature (contract + payment)
train['Contract_Payment'] = train['Contract'].astype(str) + '_' + train['PaymentMethod'].astype(str)

# Quick peek
train[['Tenure', 'TenureGroup', 'MonthlyCharges', 'HighMonthlyFlag', 'Contract', 'Contract_Ord', 'AutoPay', 'Contract_Payment','AvgChargesPerMonth']].head()


**Note:** 
- We can do the same deterministic derived feature creation to X_test (below we’ll create a pipeline so this is automatic).

### 4. Categorical Encoding Strategy
- Use One-Hot encoding for `PaymentMethod`, `TenureGroup`, and `Contract_Payment`.
- Use ordinal mapping for `Contract` (already created as `Contract_Ord`).
- Map `Gender` to binary (Male=1, Female=0) for simplicity.
- For target encoding (only if many categories), use KFold target mean encoding on TRAIN only to avoid leakage.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

# Columns
num_cols = ['Tenure', 'MonthlyCharges', 'TotalCharges', 'AvgChargesPerMonth']
num_cols = [c for c in num_cols if c in X_train.columns]  # defensive

# categorical columns
cat_cols = ['Gender','PaymentMethod', 'TenureGroup', 'Contract_Payment']

# build transformer
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
], remainder='drop')  # drop anything else


### 5. Multicollinearity & Redundancy
`TotalCharges` is mathematically derived (Tenure × MonthlyCharges) and shows very high correlation with Tenure (77%). To avoid multicollinearity, we can:
- Option A: Drop `TotalCharges` (recommended when derived).
- Option B: Keep `TotalCharges` and drop `MonthlyCharges` or `Tenure`.
- Option C: Keep only `AvgChargesPerMonth` and `Tenure` (remove direct multiply redundancy).
We will compute VIF and pick the safest choice (drop `TotalCharges` in this synthetic dataset).


In [ ]:
# Heatmap for correlation 
plt.figure(figsize=(8,6))
corr = train[['Tenure','MonthlyCharges','TotalCharges','AvgChargesPerMonth']].corr()
sns.heatmap(corr, annot=True, cmap='Blues', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# VIF (optional) — requires statsmodels
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
X_vif = train[['Tenure','MonthlyCharges','TotalCharges','AvgChargesPerMonth']]
vif_data = pd.DataFrame()
vif_data['feature'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
vif_data

During **VIF (Variance Inflation Factor)** analysis, the following results were observed:

| Feature              | VIF       |
|----------------------|-----------|
| Tenure               | 6.99      |
| MonthlyCharges       | ∞         |
| TotalCharges         | 10.17     |
| AvgChargesPerMonth   | ∞         |

🔍 R**eason for High VIF**:
- **Perfect or near-perfect multicollinearity** exists because:
  - `TotalCharges ≈ Tenure × MonthlyCharges`
  - `AvgChargesPerMonth = TotalCharges / Tenure` (essentially the same as `MonthlyCharges`)
- These relationships mean that some columns are *linear combinations* of others, leading to inflated or infinite VIF values.

✅ **Decision**:
- **Keep:**  
  - `Tenure` → Represents customer relationship duration.  
  - `MonthlyCharges` → Represents ongoing cost to the customer.  
- **Drop:**  
  - `TotalCharges` → Fully derived from `Tenure` and `MonthlyCharges`.  
  - `AvgChargesPerMonth` → Redundant with `MonthlyCharges`.

💡 **Justification**:
- Retaining **raw, base variables** (Tenure, MonthlyCharges) ensures **interpretability** and avoids information leakage.
- Removing **derived features** eliminates perfect multicollinearity, making the model more stable and coefficients more reliable.


### 6. Final Preprocessing Pipeline
- Drop `TotalCharges` and `AvgChargesPerMonth` (redundant).
- Use the pipeline (ColumnTransformer) to transform train and test consistently.
- Save the pipeline artifact for reproducibility.


In [ ]:
train.head()

In [ ]:
# Remove redundant columns from numeric list if you decide to drop it
num_cols = [c for c in num_cols if c not in ['TotalCharges' , 'AvgChargesPerMonth'] ]

# Reconstruct preprocessor with updated numeric list
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
], remainder='drop')

# Full pipeline (preprocessing + optional dimensionality reduction or model later)
from sklearn.pipeline import make_pipeline
preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit-transform on training data (we need to ensure train has created derived cols)
# Recreate derived features on X_train before fit
def create_features(data):
    data = data.copy()
    data['AvgChargesPerMonth'] = (data['TotalCharges'] / data['Tenure'])
    data['TenureGroup'] = pd.cut(data['Tenure'], bins=bins, labels=labels, include_lowest=True)
    data['HighMonthlyFlag'] = (data['MonthlyCharges'] > median_charge).astype(int)
    data['Contract_Ord'] = data['Contract'].map(contract_map)
    data['AutoPay'] = data['PaymentMethod'].isin(autopay_methods).astype(int)
    data['Contract_Payment'] = data['Contract'].astype(str) + '_' + data['PaymentMethod'].astype(str)
    return data

X_train_fe = create_features(X_train)
X_test_fe = create_features(X_test)

preprocessing_pipeline.fit(X_train_fe)
X_train_processed = preprocessing_pipeline.transform(X_train_fe)
X_test_processed = preprocessing_pipeline.transform(X_test_fe)

print("Processed train shape:", X_train_processed.shape)


In [ ]:
X_train_processed

### 7. Feature Selection & Dimensionality
After preprocessing, consider:
- Using `SelectKBest` or `SelectFromModel` (tree-based) to reduce features.
- Checking feature importances from a baseline tree (RandomForest/XGBoost) to prune weak features.
- Keeping a final feature list saved for production.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# quick baseline to get importances (use processed features)
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train_processed, y_train)

sfm = SelectFromModel(rf, threshold='median', prefit=True)
X_train_sel = sfm.transform(X_train_processed)
X_test_sel = sfm.transform(X_test_processed)
print("Selected features shape:", X_train_sel.shape)


#### Checklist — Feature Engineering Complete
- [ ] Train/test split done (stratified)
- [ ] Derived features created (TenureGroup, HighMonthlyFlag, AutoPay, Contract_Payment)
- [ ] Redundancy handled (drop `TotalCharges`,`AvgChargesPerMonth `or keep with caution)
- [ ] Encoding pipeline implemented (Label Encoder)
- [ ] Numeric scaling applied (StandardScaler)
- [ ] Feature selection tested (RandomForest + SelectFromModel)

**Next step:** use the processed training data (`X_train_processed` or selected version) to train baseline models (Logistic Regression, RandomForest, XGBoost) and compare metrics (accuracy, precision, recall, F1, ROC-AUC).


## 8. Model Training
We’ll test the following classifiers:
1. Logistic Regression
1. Support Vector Machine (SVM)
1. Decision Tree
1. Random Forest
2. AdaBoost
3. XGBoost
4. LGBM


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report

In [ ]:
from lightgbm import LGBMClassifier

# Models and params
models = {
    "Logistic Regression": (LogisticRegression(max_iter=1000), {
        "C": [0.01, 0.1, 1, 10],
        "penalty": ['l2']
    }),
    "SVM": (SVC(probability=True), {
        "C": [0.1, 1, 10],
        "kernel": ['linear', 'rbf'],
        "gamma": ['scale', 'auto']
    }),
    "Decision Tree": (DecisionTreeClassifier(), {
        "max_depth": [3, 5, 10, None],
        "min_samples_split": [2, 5, 10]
    }),
    "Random Forest": (RandomForestClassifier(), {
        "n_estimators": [100, 200],
        "max_depth": [5, 10, None]
    }),
    "AdaBoost": (AdaBoostClassifier(), {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 1]
    }),
    "XGBoost": (XGBClassifier(eval_metric='mlogloss', use_label_encoder=False), {
        "n_estimators": [100, 200],
        "max_depth": [3, 5, 7],
        "learning_rate": [0.01, 0.1, 0.2]
    }),
    "LightGBM": (LGBMClassifier(objective='binary', random_state=42), {
        "n_estimators": [100, 200],
        "max_depth": [-1, 5, 10],
        "learning_rate": [0.01, 0.1, 0.2],
        "num_leaves": [31, 50, 100]
    })
}


In [ ]:
# To store model results
results = []

best_model = None
best_auc = 0

for name, (model, params) in models.items():
    grid = GridSearchCV(model, params, cv=3, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train_sel, y_train)
    
    y_pred = grid.predict(X_test_sel)
    y_proba = grid.predict_proba(X_test_sel)
    
    auc = roc_auc_score(y_test, y_proba[:,1])
    
    # Save results for DataFrame
    results.append({
        'Model': name,
        'Best_Params': grid.best_params_,
        'ROC_AUC': auc
    })
    
    print(f"\n{name} | Best Params: {grid.best_params_} | AUC: {auc:.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
    if auc > best_auc:
        best_auc = auc
        best_model = (name, grid.best_estimator_)

# Convert to DataFrame and sort
results_df = pd.DataFrame(results).sort_values(by='ROC_AUC', ascending=False)

In [ ]:

print(f"\nBest Model: {best_model[0]} with AUC = {best_auc:.4f}")
print("\n=== Model Performance Summary (Sorted by ROC AUC) ===")
results_df


## 9. Model Evaluation
We compare models using:
- Confusion Matrix
- Classification Report
- AUC-ROC Curve


In [ ]:
final_model = best_model[1]

- Best Model is saved with the name "final_model" to use for the prediction and evaluation later 

In [ ]:
# Assuming model is already trained
y_prob = final_model.predict_proba(X_test_sel)[:, 1]  # Take probability of positive class

fpr, tpr, _ = roc_curve(y_test, y_prob)
auc_score = roc_auc_score(y_test, y_prob)

plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
plt.plot([0, 1], [0, 1], '--')  # random baseline
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# Option 1: Using seaborn heatmap
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Stay', 'Churn'], 
            yticklabels=['Stay', 'Churn'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Best Model')
plt.savefig("confusion_matrix.png", dpi=300, bbox_inches='tight')  # High quality, trimmed edges
plt.show()


In [ ]:
# Sort by ROC_AUC for better visualization
results_df = results_df.sort_values('ROC_AUC', ascending=False)

# Set seaborn style
sns.set_theme(style="whitegrid")

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='ROC_AUC', y='Model', data=results_df, palette='Blues_d')

# Add values on bars
for index, value in enumerate(results_df['ROC_AUC']):
    plt.text(value + 0.001, index, f"{value:.4f}", va='center', fontsize=10)

# Titles and labels
plt.title('Model Comparison by ROC AUC', fontsize=14)
plt.xlabel('ROC AUC Score')
plt.ylabel('Model')
plt.xlim(0.48, 0.53)  # zoom in for clarity
plt.tight_layout()
plt.savefig("barplot of models.png", dpi=300, bbox_inches='tight')  # High quality, trimmed edges
# Show plot
plt.show()


## 10. Conclusion
- **Best Model:** XGBoost achieved higher AUC-ROC compared to Logistic Regression.
- Customers on month-to-month contracts have a higher churn rate.
- Higher monthly charges correlate with higher churn.

## 11. Recommendations
- Offer loyalty discounts for month-to-month customers.
- Investigate reasons for high charges leading to churn.
- Consider targeted marketing campaigns for high-risk customers.
